# Это код для постоянного мониторинга страницы

In [5]:
# Go to the Google home page
# from bs4 import BeautifulSoup

# # url = "https://www.ozon.ru/product/elektrogril-kitfort-kt-1645-150561428/?avtc=1&avte=4&avts=1671783454&sh=_vLQ0bjDXw#section-description--offset-140"
# # url ='https://ya.ru/'
# url = 'https://global.wildberries.ru/product?card=49942341'

# def intercept(request):
#     print(request)
#     # soup = BeautifulSoup(request.date, 'html.parser')
#     # print(soup.find_all(attrs={"class":{"reviews-item__comment"}}))
#     # print(soup.find_all('title'))
#     # request.url
#     # print(request)

# driver = webdriver.Chrome()
# driver.request_interceptor = intercept
# driver.get(url)

# # time.sleep(3)

In [75]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np
import re


path_wb = 'data/wb/'
path_ozone = 'data/ozone/'

# WB

In [7]:
# from seleniumwire import webdriver  # Import from seleniumwire
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# import time

# Create a new instance of the Chrome driver
options = webdriver.ChromeOptions()
# options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
# options.headless = True # открывает браузер в фоне !!! перестает получать отзывы
browser = webdriver.Chrome(options=options)
url = 'https://global.wildberries.ru/product?card=49942341'
browser.get(url)

# Нажимает кнопку "Показать еще"
f = True
while f:
    time.sleep(3)
    try:
        btn = browser.find_element(By.CLASS_NAME, 'btn--show-more').click()
    except:
        f = False

# print(len(btn))
time.sleep(10) #productName
productName = browser.find_element(By.CLASS_NAME, 'product__title')
article = browser.find_element(By.CLASS_NAME, 'article__wrapper')
reviews = browser.find_elements(By.CLASS_NAME, 'reviews-item__comment')
stars = browser.find_elements(By.CLASS_NAME, 'stars') #data-star
# for el in browser.find_elements(By.CLASS_NAME, 'reviews-item__comment'):
#     print(el.text)
print(f'Загружено {len(reviews)} отзывов')
# browser.find_elements(By.TAG_NAME , 'div')
# browser.find_elements('reviews-item__comment')

Загружено 1000 отзывов


    Достаю текст озывов и звезды

In [8]:
reviews_items = browser.find_elements(By.CLASS_NAME, 'reviews-item')
reviews = [x.find_element(By.CLASS_NAME, 'reviews-item__comment').text for x in reviews_items]
stars = [x.find_element(By.CLASS_NAME, 'stars').get_attribute('data-star') for x in reviews_items]

    Сохраняю отзывы в файл

In [10]:


df = pd.DataFrame({'Отзывы': reviews,
                  'Звузды': stars,
                  'original': reviews})
df['Отзывы'] = df['Отзывы'].apply(lambda x: re.sub('[^A-Za-z0-9А-Яа-яЁё ,]+', '', x.lower().replace('ё', 'е')))
display(df.tail())
# file_name = article.text.replace(' ', '_').replace(':', '')+'.csv'
file_name = re.sub('[^A-Za-z0-9А-Яа-я ]+', '', article.text)
df.to_csv(path_wb + file_name + '.csv', index=True, index_label='index')
# display(df['Отзывы'].apply(lambda x: re.sub('[^A-Za-z0-9А-Яа-я ]+', '', x)))

,Отзывы,Звузды,original
995,"вилка хлипкая,но 92 р можно потерпеть все раб...",5,"Вилка хлипкая,но 92 р можно потерпеть 😅 всё ра..."
996,"работает,вилка очень хрупкая,из розетки вывали...",2,"Работает,вилка очень хрупкая,из розетки вывали..."
997,работает отлично,5,Работает отлично!
998,"заказала два,оба работают",5,"Заказала два,оба работают."
999,"отличный удлинитель, пришел в рабочем состояни...",5,"Отличный удлинитель, пришёл в рабочем состояни..."


### Работа с данными из файла

In [11]:
file_name = 'Артикул 49942341'
df = pd.read_csv(path_wb + file_name + '.csv', index_col='index')
df.tail(3)

,Отзывы,Звузды,original
index,,,
997,работает отлично,5,Работает отлично!
998,"заказала два,оба работают",5,"Заказала два,оба работают."
999,"отличный удлинитель, пришел в рабочем состояни...",5,"Отличный удлинитель, пришёл в рабочем состояни..."


    Анализ популярности слов в отзывах
* результат в файл _w.csv

In [13]:
# pd.Series(words).value_counts().sort_values(ascending=False)
words = dict()
for el in df['Отзывы'].values:
    temp = el.replace(',', ' ').split(' ')
    for i, t in enumerate(temp):
        # print(t, t not in words, words)
        if i>0: # Ищет предлоги перед словом
            predlogs = ['не', 'все']
            for predlog in predlogs:
                if temp[i-1] == predlog:
                    t = predlog + ' ' + t
        # Фильтрую слова
        if t not in ['']:
            # Добавляю слова, которые прошли фильтр
            if t in words.keys():
                words[t] = words[t] + 1
            else:
                words[t] = 1
top_words = pd.Series(words).sort_values(ascending=False)
top_words.to_csv(path_wb + file_name + '_w' + '.csv')
top_words.head(5)


все             468
удлинитель      335
работает        233
спасибо         202
все работает    191
dtype: int64

    Статистика по словарю
* результат в файл _p.csv
* результат в файл _n.csv

In [14]:
positiv = ['хорошо', 'быстро', 'все работает', 'спасибо', 'отлично', 'хороший', 'работает']
negativ = ['отвалился', 'вспыхнул', 'повреждений','не работает']

rez_positiv = {}
rez_negativ = {}
for el in positiv:
    rez_positiv[el] = df['Отзывы'].apply(lambda x: el in x).sum()
for el in negativ:
    rez_negativ[el] = df['Отзывы'].apply(lambda x: el in x).sum()

s_rez_positiv = pd.Series(rez_positiv)
s_rez_negativ = pd.Series(rez_negativ)
s_rez_positiv.to_csv(path_wb + file_name + '_p' + '.csv')
s_rez_negativ.to_csv(path_wb + file_name + '_n' + '.csv')
display(s_rez_positiv)
display(s_rez_negativ)

хорошо           93
быстро           93
все работает    202
спасибо         220
отлично         120
хороший         128
работает        449
dtype: int64

отвалился       1
вспыхнул        2
повреждений    13
не работает     7
dtype: int64

# Для проверки на ботов
https://bot.sannysoft.com/

# FF
не доделал

In [ ]:
# # Create a new instance of the Chrome driver
# from selenium.webdriver.firefox.options import Options
# # options = Options()
# options = webdriver.FirefoxOptions()
# options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
# options.set_preference('dom.webdriver.enabled', False)


# browser = webdriver.Firefox(options=options)
# url = 'https://global.wildberries.ru/product?card=49942341'
# browser.get(url)
# # Кнопка еще отзывы
# # btn--show-more btn
# # f = True
# # while f:
# #     time.sleep(3)
# #     try:
# #         btn = browser.find_element(By.CLASS_NAME, 'btn--show-more').click()
# #     except:
# #         f = False

# # print(len(btn))
# time.sleep(1)
# reviews = browser.find_elements(By.CLASS_NAME, 'reviews-item__comment')
# # for el in browser.find_elements(By.CLASS_NAME, 'reviews-item__comment'):
# #     print(el.text)
# print(len(reviews))
# # browser.find_elements(By.TAG_NAME , 'div')
# # browser.find_elements('reviews-item__comment')